In [1]:
from utils import *
check_tensorflow_gpu()

proxy = SpatiotemporalCO2()

Checking Tensorflow Version:
Tensorflow built with CUDA? True
Tensorflow version: 2.10.0
# GPU available: 1
CUDA: 64_112 | cuDNN: 64_8
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
proxy.load_data()
proxy.process_data()

X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
normalized - X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
Train - X: (750, 64, 64, 4) | y: (750, 60, 64, 64, 2)
Test  - X: (250, 64, 64, 4) | y: (250, 60, 64, 64, 2)


In [3]:
model = proxy.make_model()
fit   = proxy.train()

# Parameters: 1,116,630


In [ ]:
y_pred = model(proxy.X_train[:3])
print(y_pred.shape)

In [ ]:
fig, axs = plt.subplots(3, 12, figsize=(20,6))
for i in range(3):
    for j in range(12):
        axs[i,j].imshow(y_pred[i,j*5,:,:,-1], 'jet')
        axs[i,j].set(xticks=[], yticks=[])
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 12, figsize=(20,6))
for i in range(3):
    for j in range(12):
        axs[i,j].imshow(proxy.y_train[i,j*5,:,:,-1], 'jet')
        axs[i,j].set(xticks=[], yticks=[])
plt.show()